In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/trainmohshardness.csv')

In [ ]:
df

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10402,10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8
10405,10405,38.0,1.553160,12.666667,4.666667,26.621687,11.290033,2.743333,1.756667,0.980000,0.486507,0.77755,6.0


In [ ]:
df.columns

Index(['id', 'allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average', 'Hardness'],
      dtype='object')

In [ ]:
df.dtypes

id                         int64
allelectrons_Total       float64
density_Total            float64
allelectrons_Average     float64
val_e_Average            float64
atomicweight_Average     float64
ionenergy_Average        float64
el_neg_chi_Average       float64
R_vdw_element_Average    float64
R_cov_element_Average    float64
zaratio_Average          float64
density_Average          float64
Hardness                 float64
dtype: object

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load your dataset
# Assuming your CSV file is named 'your_dataset.csv'


# Separate features and target variable
X = df.drop('Hardness', axis=1)
y = df['Hardness']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = LinearRegression()

# Define the parameter grid for the grid search
param_grid = {
    'fit_intercept': [True, False],
    'positive': [True, False],
}

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Mean Squared Error on Test Set: {mse}")


Best Parameters: {'fit_intercept': True, 'positive': False}
Mean Squared Error on Test Set: 2.105345997105267


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Load your dataset
# Assuming your CSV file is named 'your_dataset.csv'


# Separate features and target variable
X = df.drop('Hardness', axis=1)
y = df['Hardness']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = RandomForestRegressor(random_state=42)

# Fit the model to get feature importances
model.fit(X_train, y_train)

# Display feature importances
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("Feature Importances:")
print(feature_importances)

# Select features based on importance
sfm = SelectFromModel(model, threshold=0.1)  # You can adjust the threshold
sfm.fit(X_train, y_train)

# Transform the data to keep only important features
X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

# Now, you can use the selected features for your model
# For example, you can use the same grid search approach as before:

# Define the model
selected_model = LinearRegression()

# Create the GridSearchCV object
grid_search = GridSearchCV(selected_model, param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to the data with selected features
grid_search.fit(X_train_selected, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Mean Squared Error on Test Set: {mse}")


Feature Importances:
                  Feature  Importance
10        zaratio_Average    0.209793
3    allelectrons_Average    0.119357
6       ionenergy_Average    0.109681
9   R_cov_element_Average    0.088252
11        density_Average    0.072528
4           val_e_Average    0.062858
0                      id    0.061483
5    atomicweight_Average    0.061404
7      el_neg_chi_Average    0.059630
2           density_Total    0.058764
1      allelectrons_Total    0.049789
8   R_vdw_element_Average    0.046461
Best Parameters: {'fit_intercept': True, 'positive': False}
Mean Squared Error on Test Set: 2.274995180374954


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import median_absolute_error

# Use the selected features from the feature importances
X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

# Define the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model on the selected features
rf_model.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test_selected)

# Calculate Median Absolute Error
medae_rf = median_absolute_error(y_test, y_pred_rf)

print(f"Median Absolute Error of RandomForestRegressor on Test Set: {medae_rf}")


Median Absolute Error of RandomForestRegressor on Test Set: 0.7332035714285721
